In [1]:
# https://stackoverflow.com/questions/45230448/how-to-get-reproducible-result-when-running-keras-with-tensorflow-backend

import numpy as np
import tensorflow as tf
import random as rn

import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(12345)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(1234)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [2]:
import skimage.data
import skimage.transform
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore')

plt.rcParams['figure.figsize'] = (7,7) # Make the figures a bit bigger

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
from keras.losses import sparse_categorical_crossentropy, categorical_crossentropy
from keras.initializers import glorot_uniform, he_normal
from keras.wrappers.scikit_learn import KerasClassifier
from keras.preprocessing.image import ImageDataGenerator, array_to_img

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, LeaveOneOut, KFold, cross_val_score,cross_val_predict
from sklearn.neighbors import KNeighborsClassifier 

In [3]:


import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt



# from tensorflow.python.framework import ops
import warnings


#code tutorial
#https:////github.com//PacktPublishing//Practical-Convolutional-Neural-Networks//blob//master//Chapter03//Traffic//custom.py

Train_IMAGE_DIR = "C://New folder//OneDrive//ai class//Final Project//BelgiumTSC_Training//"
Test_IMAGE_DIR = "C://New folder//OneDrive//ai class//Final Project//BelgiumTSC_Testing//"

# Train_IMAGE_DIR = 'C:/Users/bipul/niva/BelgiumTSC_Training/'
# Test_IMAGE_DIR = 'C:/Users/bipul/niva/BelgiumTSC_Testing/'

def load_data(data_dir):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all the subdirectories of the data folder (i.e. traing or test). Each folder represents an unique label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    
    # Iterate for loop through the label directories and collect the data in two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(".ppm")]

        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels

# Load training and testing datasets.
# train_data_dir = os.path.join(Train_IMAGE_DIR, "Training")
# test_data_dir = os.path.join(Test_IMAGE_DIR, "Testing")



images1, labels1 = load_data(Train_IMAGE_DIR)
images2, labels2 = load_data(Test_IMAGE_DIR)

# images1, labels1 = load_data(train_data_dir)
# images2, labels2 = load_data(test_data_dir)

#combine training and testing data

# images = images1 + images2
# labels = labels1 + labels2

# #combine training and testing data

images = images1 + images2
labels = labels1 + labels2

"Resize images"
images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

In [5]:

"Resize images"
images32 = [skimage.transform.resize(image, (32, 32), mode='constant')
                for image in images]

# "convert from list to array"
labels_array = np.array(labels)
# images_array = np.array(images32)

"scale images"
images_array = np.array(images32)
images_array = images_array.astype('float32') / 255.

# "Reshape the image data into rows"
# # images_array = np.reshape(images_array, (images_array.shape[0], -1))

"convert labels to categorical"
# labels_cat = np_utils.to_categorical(labels_array, 62)


'convert labels to categorical'

In [15]:
"split data into training and testing - 2-tier scheme"

(X_train, X_test, y_train, y_test) = train_test_split(images_array,labels, test_size=0.25, random_state=42)

In [19]:
X_train.shape

(5321, 32, 32, 3)

In [9]:

def create_model(num_neuron_l1,num_neuron_l2,activ_fn,wt_initial,drop_out):
    
    model = Sequential()
    
    model.add(Dense(num_neuron_l1,input_shape=(3072,), kernel_initializer=wt_initial))
    
    model.add(Activation(activ_fn))
    model.add(Dropout(drop_out))
    model.add(Dense(num_neuron_l2, kernel_initializer=wt_initial))
    model.add(Activation(activ_fn))
    model.add(Dropout(drop_out))
    model.add(Dense(62,kernel_initializer=wt_initial))
    model.add(Activation('softmax'))    
    
    model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy']) 
              
    return model


In [20]:
"Reshape the image data into rows"
X_train_reshape = np.reshape(X_train, (X_train.shape[0], -1))
X_test_reshape = np.reshape(X_test, (X_test.shape[0], -1))
X_train_reshape.shape

(5321, 3072)

In [21]:
"create a data generator"

datagen = ImageDataGenerator(featurewise_center=True,
                             horizontal_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

In [22]:
def data_augment(img):
    
    img_aug=next(datagen.flow(img, batch_size=1))
    
    return(img_aug)

In [23]:
#tutorial
#https://towardsdatascience.com/my-tryst-with-deep-learning-german-traffic-data-set-with-keras-87970dfb18b7
"create equal sample size for each class"


labels_array= np.asarray(labels)

X_train_aug = []
y_train_aug = []
X_aug_1 = []
Y_aug_1 = []



for i in range(0,62):
    
    class_records=np.where(labels_array==i)[0].size
    max_records=200
    samples=images_array[np.where(labels_array==i)[0]]  
    X_aug=[]
    Y_aug=[i]*max_records
    for x in range(max_records):
        img=samples[x % class_records]
        img=img.reshape((1,) + img.shape)
        trans_img=data_augment(img)
        trans_img=trans_img.reshape(32,32,3)
        X_aug.append(trans_img)
    X_train_aug = X_train_aug + X_aug
    y_train_aug = y_train_aug + Y_aug

    Y_aug_1 = Y_aug_1 + Y_aug
    X_aug_1 = X_aug_1 + X_aug



In [24]:
"convert from list to array"

new_label= np.asarray(Y_aug_1)
new_data= np.asarray(X_aug_1)


In [30]:
new_datas=np.reshape(new_data, ( new_data.shape[0], -1))

In [32]:
"divide data into 4 folds"

kf = KFold(n_splits=4)
fold1, fold2, fold3, fold4 = kf.split(new_datas)

xtrain1,xtest1=new_datas[fold1[0]],new_datas[fold1[1]]
ytrain1,ytest1=new_label[fold1[0]],new_label[fold1[1]]

xtrain2,xtest2=new_datas[fold2[0]],new_datas[fold2[1]]
ytrain2,ytest2=new_label[fold2[0]],new_label[fold2[1]]

xtrain3,xtest3=new_datas[fold3[0]],new_datas[fold3[1]]
ytrain3,ytest3=new_label[fold3[0]],new_label[fold3[1]]

xtrain4,xtest4=new_datas[fold4[0]],new_datas[fold4[1]]
ytrain4,ytest4=new_label[fold4[0]],new_label[fold4[1]]




In [27]:
model_ann=create_model(500,500,'elu','glorot_uniform',0.3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [33]:
model_ann.fit(x=xtrain1,y= ytrain1, validation_data=(xtest1, ytest1),epochs=10, batch_size=128, shuffle=False, verbose=0)
    
    
    

Instructions for updating:
Use tf.cast instead.


In [34]:
pred= model_ann.predict(xtest1)

In [35]:
ann_pred = np.argmax(np.round(pred), axis=1)

In [36]:
ytest1

array([ 0,  0,  0, ..., 15, 15, 15])

In [38]:
from imblearn.metrics import sensitivity_specificity_support
report2= sensitivity_specificity_support(ytest1,ann_pred,average='weighted')
print(report2)

(0.06451612903225806, 0.9354838709677419, None)


In [39]:
report1= classification_report(ytest1,ann_pred)
print(report1)

              precision    recall  f1-score   support

           0       0.06      1.00      0.12       200
           1       0.00      0.00      0.00       200
           2       0.00      0.00      0.00       200
           3       0.00      0.00      0.00       200
           4       0.00      0.00      0.00       200
           5       0.00      0.00      0.00       200
           6       0.00      0.00      0.00       200
           7       0.00      0.00      0.00       200
           8       0.00      0.00      0.00       200
           9       0.00      0.00      0.00       200
          10       0.00      0.00      0.00       200
          11       0.00      0.00      0.00       200
          12       0.00      0.00      0.00       200
          13       0.00      0.00      0.00       200
          14       0.00      0.00      0.00       200
          15       0.00      0.00      0.00       100

   micro avg       0.06      0.06      0.06      3100
   macro avg       0.00   

In [40]:
report3=accuracy_score(ytest1, ann_pred)
print(report3)

0.06451612903225806


In [ ]:
def get_eval(X_train, X_test, y_train, y_test):
    
#     scaler= StandardScaler(with_std=False)
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
  
    cnn2.fit(x=X_train,y= y_train, validation_data=(X_test, y_test),epochs=30, batch_size=128, shuffle=False, verbose=0)
    preddd= cnn2.predict(X_test)
    cnn_pred = np.argmax(np.round(preddd), axis=1)
    
    
    report1= classification_report(y_test,cnn_pred)
    print(report1)
    report2= sensitivity_specificity_support(y_test,cnn_pred,average='weighted')
    print(report2)
    report3=accuracy_score(X_test, y_test)
    preint(report3)
    
    

In [ ]:
score2 = get_eval(xtrain2,xtest2,ytrain2,ytest2)
score3 = get_eval(xtrain3,xtest3,ytrain3,ytest3)
score4 = get_eval(xtrain4,xtest4,ytrain4,ytest4)